# Monte Carlo RSM Simulator

This notebook demonstrates how to use the Monte Carlo simulator to analyze RSM (Replicated State Machine) deployments. We'll explore:

1. Setting up node configurations with failure distributions
2. Running single simulations
3. Running Monte Carlo simulations for statistical analysis
4. Visualizing availability and time-to-data-loss distributions

In [ ]:
# Imports
from powder.simulation import (
    # Time units
    Seconds, hours, days, minutes,
    # Distributions
    Exponential, Weibull, Normal, Uniform, Constant,
    # Node
    NodeConfig, NodeState,
    # Network
    NetworkConfig, NetworkState,
    # Cluster
    ClusterState,
    # Strategy
    NoOpStrategy, SimpleReplacementStrategy, NodeReplacementStrategy,
    # Protocol
    LeaderlessUpToDateQuorumProtocol, LeaderlessMajorityAvailableProtocol, RaftLikeProtocol,
    # Simulator
    Simulator,
)
from powder.monte_carlo import (
    run_monte_carlo, MonteCarloConfig, MonteCarloRunner,
    run_monte_carlo_converged, ConvergenceCriteria, ConvergenceMetric,
    estimate_required_runs,
)
from powder.graphing_utils import (
    make_pdf_histogram,
    make_cdf_plot,
    make_availability_boxplot,
    make_time_to_loss_comparison,
    make_multi_cdf_plot,
)
import numpy as np

## 1. Define Node Configuration

Each node has:
- **region**: Geographic location
- **cost_per_hour**: Dollar cost to run the node
- **failure_dist**: Distribution for time until transient failure
- **recovery_dist**: Distribution for time to recover from failure
- **data_loss_dist**: Distribution for time until permanent data loss (e.g., disk failure)
- **log_replay_rate_dist**: Distribution for log replay rate (committed-data units replayed per second of wall time)
- **snapshot_download_time_dist**: Distribution for wall-clock time to download a full snapshot
- **spawn_dist**: Distribution for time to spawn a new replacement node

Snapshot interval and commit rate are configured on the Protocol, not on individual nodes.

In [ ]:
# Create a realistic node configuration
def make_standard_node_config(region: str = "us-east") -> NodeConfig:
    return NodeConfig(
        region=region,
        cost_per_hour=0.50,  # $0.50/hour per node
        # Transient failures: ~1 per week on average
        failure_dist=Exponential(rate=1 / days(7)),
        # Recovery time: 5-15 minutes
        recovery_dist=Uniform(low=minutes(5), high=minutes(15)),
        # Permanent data loss: ~1 per year (disk failure rate)
        data_loss_dist=Exponential(rate=1 / days(365)),
        # Log replay rate: node can replay 3 units of committed data per 1 second of wall time
        log_replay_rate_dist=Constant(3.0),
        # Snapshot download time: 5 minutes to download a full snapshot
        snapshot_download_time_dist=Constant(minutes(5)),
        # Spawn time: 10-20 minutes to provision a new node
        spawn_dist=Uniform(low=minutes(1), high=minutes(5)),
    )

node_config = make_standard_node_config()
print(f"Node config: {node_config.region}, ${node_config.cost_per_hour}/hr")

## 2. Create Initial Cluster

A cluster consists of multiple nodes. For a 3-node cluster, we need a quorum of 2 nodes to commit new requests.

In [ ]:
def create_cluster(num_nodes: int = 3, regions: list[str] | None = None) -> ClusterState:
    """Create a fresh cluster with the specified number of nodes."""
    if regions is None:
        regions = ["us-east", "us-west", "eu-central"]
    
    nodes = {}
    for i in range(num_nodes):
        region = regions[i % len(regions)]
        config = make_standard_node_config(region)
        nodes[f"node{i}"] = NodeState(
            node_id=f"node{i}",
            config=config,
        )
    
    return ClusterState(
        nodes=nodes,
        network=NetworkState(),
        target_cluster_size=num_nodes,
    )

cluster = create_cluster(3)
print(f"Created cluster: {cluster}")
print(f"Quorum size: {cluster.quorum_size()}")
print(f"Can commit: {cluster.can_commit()}")

## 3. Run a Single Simulation

Let's run a simulation for 30 days and observe the behavior.

In [ ]:
# Run simulation for 30 days with no intervention strategy
cluster = create_cluster(3)
simulator = Simulator(
    initial_cluster=cluster,
    strategy=NoOpStrategy(),
    protocol=LeaderlessUpToDateQuorumProtocol(),
    seed=42,
    log_events=True,  # Keep event log for debugging
)

result = simulator.run_for(days(1000))

print(f"Simulation ended: {result.end_reason}")
print(f"End time: {result.end_time / 86400:.2f} days")
print(f"Availability: {result.metrics.availability_fraction() * 100:.2f}%")
print(f"Total cost: ${result.metrics.total_cost:.2f}")
print(f"Events processed: {len(result.event_log)}")

In [ ]:
# Show all node_sync_complete events
from powder.simulation.events import EventType
sync_events = [e for e in result.event_log if e.event_type == EventType.NODE_SYNC_COMPLETE]
print(f"Node sync complete events: {len(sync_events)}")
for event in sync_events:
    print(f"  t={event.time/3600:.2f}h ({event.time/86400:.1f}d): {event.event_type.value} on {event.target_id}")

## 4. Run Monte Carlo Simulations

To get statistical properties, we run many simulations and aggregate the results.

In [ ]:
# Run 100 simulations, each for 1 year, with NoOp strategy
results_no_op = run_monte_carlo(
    cluster_factory=lambda: create_cluster(3),
    strategy_factory=NoOpStrategy,
    protocol=LeaderlessUpToDateQuorumProtocol(),
    num_simulations=100,
    max_time=days(365),
    stop_on_data_loss=True,
    seed=42,
)

print(results_no_op.summary())

In [ ]:
# Now run with SimpleReplacementStrategy that spawns replacements for failed nodes
def make_replacement_strategy():
    return SimpleReplacementStrategy(
        default_node_config=make_standard_node_config(),
        scale_down_threshold=3,  # Only scale down if 3+ nodes fail simultaneously
    )

results_with_replacement = run_monte_carlo(
    cluster_factory=lambda: create_cluster(3),
    strategy_factory=make_replacement_strategy,
    protocol=LeaderlessUpToDateQuorumProtocol(),
    num_simulations=100,
    max_time=days(365),
    stop_on_data_loss=True,
    seed=42,
)

print(results_with_replacement.summary())

### 4a. Node Replacement Strategy — Timeout-Based Replacement

The `NodeReplacementStrategy` models the standard production protocol for handling
node failures in replicated state machines. Unlike `SimpleReplacementStrategy`
which only replaces nodes after permanent data loss, this strategy also replaces
nodes that have been **unavailable for too long** due to transient failures or
network outages.

**How it works:**
1. When a node becomes unavailable, a configurable **failure timeout** countdown begins.
2. If the node recovers before the timeout, the countdown is cancelled.
3. If the timeout fires and the node is still unavailable *and* the cluster can still
   commit, the failed node is removed and a replacement is spawned.
4. For permanent data loss, replacement is immediate — no timeout wait.

The `failure_timeout` should be set longer than the typical recovery time to avoid
unnecessary replacements for transient failures that resolve quickly.

In [ ]:
# NodeReplacementStrategy: replace nodes that are unavailable too long
# The timeout is set to 30 minutes — longer than the typical 5-15 min recovery,
# so transient failures that resolve quickly won't trigger unnecessary replacements.

def make_node_replacement_strategy():
    return NodeReplacementStrategy(
        failure_timeout=minutes(30),
        default_node_config=make_standard_node_config(),
    )

results_node_replacement = run_monte_carlo(
    cluster_factory=lambda: create_cluster(3),
    strategy_factory=make_node_replacement_strategy,
    protocol=LeaderlessUpToDateQuorumProtocol(),
    num_simulations=100,
    max_time=days(365),
    stop_on_data_loss=True,
    seed=42,
)

print("NodeReplacementStrategy (timeout-based):")
print(results_node_replacement.summary())

print("\nFor comparison, SimpleReplacementStrategy (data-loss only):")
print(results_with_replacement.summary())

In [ ]:
# Compare all three strategies
fig = make_availability_boxplot(
    results_list=[results_no_op, results_with_replacement, results_node_replacement],
    labels=["No Intervention", "Simple Replacement", "Node Replacement (30min timeout)"],
    title="Availability: No-Op vs Simple Replacement vs Node Replacement",
)
fig.show()

In [ ]:
# Single-run event log to see the replacement strategy in action
cluster = create_cluster(3)
simulator = Simulator(
    initial_cluster=cluster,
    strategy=NodeReplacementStrategy(
        failure_timeout=minutes(30),
        default_node_config=make_standard_node_config(),
    ),
    protocol=LeaderlessUpToDateQuorumProtocol(),
    seed=42,
    log_events=True,
)

result = simulator.run_for(days(365))
print(f"End reason: {result.end_reason}")
print(f"Availability: {result.metrics.availability_fraction() * 100:.2f}%")
print(f"Events: {len(result.event_log)}")

# Show replacement-related events
print("\nReplacement-related events:")
from powder.simulation.events import EventType
for event in result.event_log:
    if event.event_type in (
        EventType.NODE_DATA_LOSS,
        EventType.NODE_REPLACEMENT_TIMEOUT,
        EventType.NODE_SPAWN_COMPLETE,
    ):
        print(f"  t={event.time/86400:.2f}d: {event.event_type.value} on {event.target_id}")

## 4b. Adaptive Monte Carlo — Automatic Sample Size

Instead of guessing how many simulations to run, we can let the runner
automatically determine the required number of runs to achieve a desired
confidence level and relative error on key metrics.

The runner works in two phases:
1. **Pilot batch**: Runs `min_runs` simulations to estimate variance.
2. **Adaptive batches**: Uses the t-distribution CI formula `n = (t * σ / (ε * μ))²`
   to estimate how many more runs are needed, running them in batches until all
   target metrics converge or `max_runs` is reached.

In [ ]:
# Run adaptive Monte Carlo — converge on availability with 95% CI, 5% relative error
converged_result = run_monte_carlo_converged(
    cluster_factory=lambda: create_cluster(3),
    strategy_factory=make_replacement_strategy,
    protocol=LeaderlessUpToDateQuorumProtocol(),
    max_time=days(365),
    confidence_level=0.95,
    relative_error=0.05,  # CI half-width <= 5% of the mean
    metrics=[ConvergenceMetric.AVAILABILITY],
    stop_on_data_loss=True,
    seed=42,
    min_runs=30,
    max_runs=5000,
    batch_size=20,
)

print(converged_result.summary())

In [ ]:
# Converge on multiple metrics simultaneously
# The runner will keep going until ALL specified metrics are within tolerance.
converged_multi = run_monte_carlo_converged(
    cluster_factory=lambda: create_cluster(3),
    strategy_factory=make_replacement_strategy,
    protocol=LeaderlessUpToDateQuorumProtocol(),
    max_time=days(365),
    confidence_level=0.95,
    relative_error=0.05,
    metrics=[
        ConvergenceMetric.AVAILABILITY,
        ConvergenceMetric.COST,
        ConvergenceMetric.DATA_LOSS_PROBABILITY,
    ],
    stop_on_data_loss=True,
    seed=42,
    min_runs=30,
    max_runs=5000,
    batch_size=20,
)

print(converged_multi.summary())

In [ ]:
# You can also estimate required runs from a small pilot before committing
# to a full adaptive run. This is useful for planning compute budgets.
pilot = run_monte_carlo(
    cluster_factory=lambda: create_cluster(3),
    strategy_factory=make_replacement_strategy,
    protocol=LeaderlessUpToDateQuorumProtocol(),
    num_simulations=30,
    max_time=days(365),
    stop_on_data_loss=True,
    seed=42,
)

estimates = estimate_required_runs(
    pilot,
    ConvergenceCriteria(
        confidence_level=0.95,
        relative_error=0.02,
        metrics=[ConvergenceMetric.AVAILABILITY, ConvergenceMetric.COST],
    ),
)

for metric, n in estimates.items():
    print(f"  {metric.value}: ~{n} runs needed for 95% CI with 5% relative error")

In [ ]:
# Example: Watch convergence happen in real-time
#
# This uses a progress callback to print status after every batch,
# so you can see the estimated sample size shrink as variance stabilises.

print(f"{'Completed':>10} {'Est. Total':>11} {'Converged':>10}")
print("-" * 35)

def on_progress(completed, estimated_total, converged):
    print(f"{completed:>10} {estimated_total:>11} {'  yes' if converged else '   no':>10}")

converged_example = run_monte_carlo_converged(
    cluster_factory=lambda: create_cluster(5),
    strategy_factory=make_replacement_strategy,
    protocol=LeaderlessUpToDateQuorumProtocol(),
    max_time=days(365),
    confidence_level=0.95,
    relative_error=0.05,
    metrics=[ConvergenceMetric.AVAILABILITY, ConvergenceMetric.COST],
    stop_on_data_loss=True,
    seed=123,
    min_runs=30,
    max_runs=2000,
    batch_size=20,
    progress_callback=on_progress,
)

print()
print(f"Finished in {converged_example.total_runs} runs "
      f"(converged: {converged_example.converged})")
print()

# Show per-metric detail
for s in converged_example.metric_statuses:
    ci_lo = s.current_mean - s.ci_half_width
    ci_hi = s.current_mean + s.ci_half_width
    print(f"  {s.metric.value}:")
    print(f"    mean = {s.current_mean:.4f}  "
          f"95% CI = [{ci_lo:.4f}, {ci_hi:.4f}]  "
          f"rel_error = {s.relative_error:.4f}")

# Compare: the fixed 100-run result from earlier
print()
print(f"For comparison, the fixed 100-run result had availability "
      f"std = {results_with_replacement.availability_std()*100:.2f}% "
      f"across only {len(results_with_replacement.availability_samples)} samples.")

### Decimal-place accuracy with absolute error

The examples above use `relative_error`, which bounds the CI half-width as a
fraction of the mean (e.g., 5% of the mean). But if you want availability
accurate to a specific number of **decimal places**, use `absolute_error` instead.

| Desired accuracy | `absolute_error` value | Meaning |
|---|---|---|
| 1 decimal place on the fraction | `0.1` | mean ± 0.1 (e.g., 0.4 ± 0.1) |
| 2 decimal places on the fraction | `0.01` | mean ± 0.01 (e.g., 0.45 ± 0.01) |
| 1 decimal place on the percentage | `0.001` | mean ± 0.1% (e.g., 45.7% ± 0.1%) |
| 2 decimal places on the percentage | `0.0001` | mean ± 0.01% (e.g., 45.74% ± 0.01%) |

In [ ]:
# Get availability accurate to 2 decimal places on the fraction (±0.01)
# i.e. if the true mean is 0.4574, we want our estimate in [0.4474, 0.4674]

result_2dp = run_monte_carlo_converged(
    cluster_factory=lambda: create_cluster(5),
    strategy_factory=make_replacement_strategy,
    protocol=LeaderlessUpToDateQuorumProtocol(),
    max_time=days(365),
    confidence_level=0.95,
    absolute_error=0.01,  # ±0.01 on the availability fraction
    metrics=[ConvergenceMetric.AVAILABILITY],
    stop_on_data_loss=True,
    seed=42,
    min_runs=30,
    max_runs=10_000,
    batch_size=50,
)

status = result_2dp.metric_statuses[0]
mean = status.current_mean
hw = status.ci_half_width

print(f"Runs needed: {result_2dp.total_runs}")
print(f"Availability: {mean:.4f}  95% CI: [{mean - hw:.4f}, {mean + hw:.4f}]")
print(f"CI half-width: ±{hw:.4f}  (target: ±0.0100)")
print(f"Converged: {result_2dp.converged}")

### 4c. Mean Time to Data Loss (MTTDL) — Convergence

To estimate **mean time to data loss** with a confidence interval, set
`max_time=None` so each simulation runs indefinitely until data loss occurs.
This guarantees every run contributes a sample, making convergence
straightforward.

The adaptive runner will keep adding batches until the CI on MTTDL is
within the specified tolerance (relative or absolute).

In [ ]:
# Converge on mean time to data loss with 95% CI, 5% relative error.
# max_time=None means each run continues until data loss — no truncation.

mttdl_result = run_monte_carlo_converged(
    cluster_factory=lambda: create_cluster(3),
    strategy_factory=make_replacement_strategy,
    protocol=LeaderlessUpToDateQuorumProtocol(),
    max_time=None,            # run each sim until data loss occurs
    stop_on_data_loss=True,
    confidence_level=0.95,
    relative_error=0.05,      # CI half-width ≤ 5% of the mean
    metrics=[ConvergenceMetric.MEAN_TIME_TO_DATA_LOSS],
    seed=42,
    min_runs=30,
    max_runs=5000,
    batch_size=20,
)

print(mttdl_result.summary())

# Also available programmatically:
mc = mttdl_result.results
ci = mc.ci_time_to_actual_loss()
if ci is not None:
    print(f"\nMTTDL: {mc.mean_time_to_actual_loss() / 86400:.1f} days")
    print(f"  95% CI: [{ci[0] / 86400:.1f}, {ci[1] / 86400:.1f}] days")
    print(f"  Std:    {mc.std_time_to_actual_loss() / 86400:.1f} days")
    print(f"  Runs:   {len(mc.time_to_actual_loss_samples_filtered())}")

## 5. Visualize Results

### 5.1 Availability Distribution

In [ ]:
# PDF histogram of availability
avail_samples = [a * 100 for a in results_with_replacement.availability_samples]
fig = make_pdf_histogram(
    samples=avail_samples,
    title="Availability Distribution (with Replacement Strategy)",
    x_label="Availability (%)",
    bins=30,
)
fig.show()

In [ ]:
# Compare availability between strategies
fig = make_availability_boxplot(
    results_list=[results_no_op, results_with_replacement],
    labels=["No Intervention", "With Replacement"],
    title="Availability Comparison: No Intervention vs Replacement Strategy",
)
fig.show()

### 5.2 Time to Data Loss Distribution

In [ ]:
# CDF of time to actual data loss (NoOp strategy)
loss_times_days = [t / 86400 for t in results_no_op.time_to_actual_loss_samples_filtered()]

if loss_times_days:
    fig = make_cdf_plot(
        samples=loss_times_days,
        title="Time to Data Loss CDF (No Intervention)",
        x_label="Days",
        show_percentiles=[50, 90, 99],
    )
    fig.show()
else:
    print("No data loss events occurred in the simulations")

In [ ]:
# PDF histogram of time to data loss
if loss_times_days:
    fig = make_pdf_histogram(
        samples=loss_times_days,
        title="Time to Data Loss PDF (No Intervention)",
        x_label="Days",
        bins=30,
    )
    fig.show()

## 6. Parameter Sensitivity Analysis

Let's see how the number of nodes affects availability and time to data loss.

In [ ]:
# Compare 3, 5, and 7 node clusters
results_by_size = {}

for num_nodes in [3, 5, 7]:
    results = run_monte_carlo(
        cluster_factory=lambda n=num_nodes: create_cluster(n),
        strategy_factory=make_replacement_strategy,
        protocol=LeaderlessUpToDateQuorumProtocol(),
        num_simulations=50,
        max_time=days(365),
        stop_on_data_loss=True,
        seed=42,
    )
    results_by_size[num_nodes] = results
    print(f"\n{num_nodes}-node cluster:")
    print(results.summary())

In [ ]:
# Compare availability across cluster sizes
fig = make_availability_boxplot(
    results_list=list(results_by_size.values()),
    labels=[f"{n} nodes" for n in results_by_size.keys()],
    title="Availability by Cluster Size",
)
fig.show()

In [ ]:
# Compare time to data loss across cluster sizes
fig = make_time_to_loss_comparison(
    results_list=list(results_by_size.values()),
    labels=[f"{n} nodes" for n in results_by_size.keys()],
    title="Time to Data Loss by Cluster Size",
    actual=True,
    time_unit="days",
)
fig.show()

## 7. Custom Failure Scenarios

Let's simulate a scenario with unreliable nodes (higher failure rates).

In [ ]:
def make_unreliable_node_config(region: str = "us-east") -> NodeConfig:
    """Node with higher failure rates."""
    return NodeConfig(
        region=region,
        cost_per_hour=0.30,  # Cheaper but less reliable
        # Transient failures: ~1 per day
        failure_dist=Exponential(rate=1 / days(1)),
        # Recovery time: 10-30 minutes (slower recovery)
        recovery_dist=Uniform(low=minutes(10), high=minutes(30)),
        # Permanent data loss: ~1 per 6 months
        data_loss_dist=Exponential(rate=1 / days(180)),
        log_replay_rate_dist=Constant(2.0),  # Slower log replay
        snapshot_download_time_dist=Constant(minutes(10)),  # Slower snapshot download
        spawn_dist=Uniform(low=minutes(15), high=minutes(30)),
    )

def create_unreliable_cluster(num_nodes: int = 5) -> ClusterState:
    nodes = {}
    for i in range(num_nodes):
        config = make_unreliable_node_config()
        nodes[f"node{i}"] = NodeState(
            node_id=f"node{i}",
            config=config,
        )
    return ClusterState(
        nodes=nodes,
        network=NetworkState(),
        target_cluster_size=num_nodes,
    )

# Run simulation with unreliable nodes
results_unreliable = run_monte_carlo(
    cluster_factory=lambda: create_unreliable_cluster(5),
    strategy_factory=lambda: SimpleReplacementStrategy(
        default_node_config=make_unreliable_node_config(),
    ),
    protocol=LeaderlessUpToDateQuorumProtocol(),
    num_simulations=100,
    max_time=days(365),
    stop_on_data_loss=True,
    seed=42,
)

print("Unreliable 5-node cluster:")
print(results_unreliable.summary())

In [ ]:
# Compare reliable vs unreliable clusters
fig = make_availability_boxplot(
    results_list=[results_by_size[5], results_unreliable],
    labels=["Reliable 5-node", "Unreliable 5-node"],
    title="Reliable vs Unreliable Nodes",
)
fig.show()

## 8. Protocol Comparison: Leaderless vs Raft

The simulator supports three consensus protocol models with different availability semantics:

- **LeaderlessUpToDateQuorumProtocol**: Any node can propose; requires a quorum of *up-to-date* nodes to commit (e.g., EPaxos, multi-decree Paxos).
- **LeaderlessMajorityAvailableProtocol**: Any node can propose; requires a quorum of *available* nodes, even if they are lagging behind (e.g., eventually-consistent systems). More permissive — lagging replicas can still participate.
- **RaftLikeProtocol**: A single leader accepts writes; if the leader fails, an election occurs and the system is **unavailable for the duration of the election**. Requires leader + quorum of up-to-date nodes (e.g., Raft, Multi-Paxos with a stable leader).

The key tradeoff: Raft-like protocols pay an availability cost during leader elections, but the leaderless up-to-date quorum protocol requires *all* quorum members to be fully caught up.

In [ ]:
# Run Monte Carlo for each protocol with the same cluster & strategy
protocols = {
    "Leaderless (up-to-date quorum)": LeaderlessUpToDateQuorumProtocol(),
    "Leaderless (majority available)": LeaderlessMajorityAvailableProtocol(),
    "Raft-like (3-8s election)": RaftLikeProtocol(
        election_time_dist=Uniform(low=Seconds(3), high=Seconds(8)),
    ),
}

protocol_results = {}
for label, protocol in protocols.items():
    result = run_monte_carlo(
        cluster_factory=lambda: create_cluster(3),
        strategy_factory=make_node_replacement_strategy,
        protocol=protocol,
        num_simulations=100,
        max_time=days(365),
        stop_on_data_loss=True,
        seed=42,
    )
    protocol_results[label] = result
    print(f"\n{label}:")
    print(result.summary())

In [ ]:
# Compare availability across protocols
fig = make_availability_boxplot(
    results_list=list(protocol_results.values()),
    labels=list(protocol_results.keys()),
    title="Availability by Consensus Protocol",
)
fig.show()

In [ ]:
# Overlay time-to-data-loss CDFs across protocols
loss_samples = []
loss_labels = []
for label, result in protocol_results.items():
    samples = [t / 86400 for t in result.time_to_actual_loss_samples_filtered()]
    if samples:
        loss_samples.append(samples)
        loss_labels.append(label)

if loss_samples:
    fig = make_multi_cdf_plot(
        samples_list=loss_samples,
        labels=loss_labels,
        title="Time to Data Loss CDF — Protocol Comparison",
        x_label="Days",
    )
    fig.show()
else:
    print("No data loss events to compare")

## 9. Network Partition Simulation

Real distributed systems face **network partitions** in addition to node failures.
The simulator supports region-level network outages via `NetworkConfig`:

- **outage_dist**: Distribution for time between network outage events
- **outage_duration_dist**: Distribution for how long each outage lasts
- **affected_regions**: Which regions can experience full network outages

When a region's network is down, all nodes in that region become **effectively unavailable**
for quorum purposes — even though the nodes themselves haven't crashed. This can cause
quorum loss if multiple nodes share a region, and forces nodes to re-sync when the
partition heals (since they fall behind on commits during the outage).

In [ ]:
# Define a network configuration with periodic regional outages
network_config = NetworkConfig(
    outage_dist=Exponential(rate=1 / days(30)),  # ~1 outage per month per region
    outage_duration_dist=Uniform(low=minutes(10), high=hours(2)),  # 10min–2hr outages
    affected_regions=["us-east", "us-west", "eu-central"],
)

# Run with network partitions enabled
results_with_partitions = run_monte_carlo(
    cluster_factory=lambda: create_cluster(3),
    strategy_factory=make_node_replacement_strategy,
    protocol=LeaderlessUpToDateQuorumProtocol(),
    num_simulations=100,
    max_time=days(365),
    stop_on_data_loss=True,
    seed=42,
    network_config=network_config,
)

print("With network partitions (~monthly per region, 10min-2hr):")
print(results_with_partitions.summary())
print("\nWithout network partitions (for comparison):")
print(results_node_replacement.summary())

In [ ]:
# Compare availability with and without network partitions
fig = make_availability_boxplot(
    results_list=[results_node_replacement, results_with_partitions],
    labels=["Node failures only", "Node failures + network partitions"],
    title="Impact of Network Partitions on Availability",
)
fig.show()

In [ ]:
# Single run with event logging to see network events in action
cluster = create_cluster(3)
simulator = Simulator(
    initial_cluster=cluster,
    strategy=make_node_replacement_strategy(),
    protocol=LeaderlessUpToDateQuorumProtocol(),
    network_config=network_config,
    seed=42,
    log_events=True,
)

result = simulator.run_for(days(365))
print(f"End reason: {result.end_reason}")
print(f"Availability: {result.metrics.availability_fraction() * 100:.2f}%")

# Show network-related events
print("\nNetwork events:")
for event in result.event_log:
    if event.event_type in (EventType.NETWORK_OUTAGE_START, EventType.NETWORK_OUTAGE_END):
        region = event.metadata.get("region", event.target_id)
        print(f"  t={event.time/86400:.2f}d: {event.event_type.value} — {region}")

## 10. Snapshot Interval & Commit Rate Sensitivity

Protocols have two key parameters that affect **node recovery time**:

- **commit_rate**: How fast data is committed (data units per second of wall time). A higher commit rate means nodes fall further behind during failures and take longer to catch up.
- **snapshot_interval**: How often nodes take snapshots (in commit-index units). With snapshots, a lagging node can download a snapshot instead of replaying the entire log from the beginning — but it pays a fixed download cost.

The recovery model works as follows:
1. If the lagging node is behind the donor's last snapshot, it must **download the snapshot** first, then replay the remaining log suffix.
2. If the node is ahead of the donor's snapshot, it replays the log directly.
3. During recovery, new commits keep arriving at `commit_rate`, so the net catch-up rate is `(log_replay_rate - commit_rate)`. If the commit rate exceeds the replay rate, the node **can never catch up**.

Let's see how these parameters affect availability.

In [ ]:
# Compare different commit rates
# Higher commit rates mean nodes fall further behind during downtime
# Recall: node log_replay_rate is 3.0 units/s, so commit_rate must be < 3.0 to catch up

commit_rate_configs = {
    "commit_rate=0.5 (low load)": LeaderlessUpToDateQuorumProtocol(commit_rate=0.5),
    "commit_rate=1.0 (default)": LeaderlessUpToDateQuorumProtocol(commit_rate=1.0),
    "commit_rate=2.5 (high load)": LeaderlessUpToDateQuorumProtocol(commit_rate=2.5),
}

commit_rate_results = {}
for label, protocol in commit_rate_configs.items():
    result = run_monte_carlo(
        cluster_factory=lambda: create_cluster(3),
        strategy_factory=make_node_replacement_strategy,
        protocol=protocol,
        num_simulations=100,
        max_time=days(365),
        stop_on_data_loss=True,
        seed=42,
    )
    commit_rate_results[label] = result
    print(f"\n{label}:")
    print(result.summary())

In [ ]:
fig = make_availability_boxplot(
    results_list=list(commit_rate_results.values()),
    labels=list(commit_rate_results.keys()),
    title="Availability by Commit Rate (log_replay_rate=3.0)",
)
fig.show()

In [ ]:
# Compare snapshot intervals
# With snapshot_interval=0, there is no log truncation — a lagging node replays the full log.
# With snapshots, the node downloads a snapshot and then replays only the suffix.

snapshot_configs = {
    "No snapshots": LeaderlessUpToDateQuorumProtocol(
        commit_rate=1.0, snapshot_interval=0,
    ),
    "Snapshot every 3600 units": LeaderlessUpToDateQuorumProtocol(
        commit_rate=1.0, snapshot_interval=3600,
    ),
    "Snapshot every 600 units": LeaderlessUpToDateQuorumProtocol(
        commit_rate=1.0, snapshot_interval=600,
    ),
}

snapshot_results = {}
for label, protocol in snapshot_configs.items():
    result = run_monte_carlo(
        cluster_factory=lambda: create_cluster(3),
        strategy_factory=make_node_replacement_strategy,
        protocol=protocol,
        num_simulations=100,
        max_time=days(365),
        stop_on_data_loss=True,
        seed=42,
    )
    snapshot_results[label] = result
    print(f"\n{label}:")
    print(result.summary())

In [ ]:
fig = make_availability_boxplot(
    results_list=list(snapshot_results.values()),
    labels=list(snapshot_results.keys()),
    title="Availability by Snapshot Interval",
)
fig.show()

## 11. Cost Analysis

The simulator tracks infrastructure costs throughout each run. Every node
(including nodes being provisioned) is billed at its `cost_per_hour` rate.
Replacement strategies that spawn new nodes increase total cost, while
strategies that scale down reduce it.

Let's visualize how costs vary across different configurations and strategies.

In [ ]:
# Cost distribution: NoOp vs SimpleReplacement vs NodeReplacement
fig = make_pdf_histogram(
    samples=results_no_op.cost_samples,
    title="Cost Distribution — No Intervention (3-node, 1 year)",
    x_label="Total Cost ($)",
    bins=30,
)
fig.show()

fig = make_pdf_histogram(
    samples=results_node_replacement.cost_samples,
    title="Cost Distribution — Node Replacement Strategy (3-node, 1 year)",
    x_label="Total Cost ($)",
    bins=30,
    color="coral",
)
fig.show()

In [ ]:
# Overlay cost CDFs across strategies for direct comparison
cost_samples_list = [
    results_no_op.cost_samples,
    results_with_replacement.cost_samples,
    results_node_replacement.cost_samples,
]
cost_labels = [
    f"No Intervention (mean=${results_no_op.cost_mean():,.0f})",
    f"Simple Replacement (mean=${results_with_replacement.cost_mean():,.0f})",
    f"Node Replacement (mean=${results_node_replacement.cost_mean():,.0f})",
]

fig = make_multi_cdf_plot(
    samples_list=cost_samples_list,
    labels=cost_labels,
    title="Cost CDF — Strategy Comparison (3-node, 1 year)",
    x_label="Total Cost ($)",
)
fig.show()

In [ ]:
# Cost vs availability tradeoff across cluster sizes
print(f"{'Config':<25} {'Availability':>14} {'Mean Cost':>12} {'Cost/9':>10}")
print("-" * 65)

for num_nodes, result in results_by_size.items():
    avail = result.availability_mean()
    cost = result.cost_mean()
    nines = -np.log10(1 - avail) if avail < 1.0 else float("inf")
    cost_per_nine = cost / nines if nines > 0 and nines != float("inf") else float("nan")
    print(
        f"{num_nodes}-node cluster{'':<13} "
        f"{avail*100:>13.2f}% "
        f"${cost:>10,.0f} "
        f"${cost_per_nine:>8,.0f}"
    )

## Summary

This simulator allows you to:

1. **Model RSM deployments** with realistic node configurations
2. **Define failure distributions** (exponential, Weibull, uniform, etc.)
3. **Compare consensus protocols** — leaderless (up-to-date quorum, majority available) and leader-based (Raft-like with election downtime)
4. **Simulate network partitions** — region-level outages that make nodes unavailable without crashing them
5. **Tune protocol parameters** — commit rate and snapshot interval to understand recovery dynamics
6. **Implement custom strategies** for handling failures (no-op, simple replacement, timeout-based replacement)
7. **Run Monte Carlo simulations** with fixed sample sizes or adaptive convergence for high-confidence results
8. **Analyze costs** — infrastructure cost distributions and cost/availability tradeoffs
9. **Visualize results** with PDFs, CDFs, box plots, multi-CDF overlays, and comparison charts

Key metrics computed:
- **Availability**: Fraction of time the system can accept commits
- **Time to potential data loss**: When quorum is first lost
- **Time to actual data loss**: When all up-to-date nodes fail
- **Total cost**: Accumulated infrastructure costs (including provisioning)
- **Cost per nine**: Dollar cost for each nine of availability